In [11]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [12]:
#this version will delete const if it is not statistically significant
def backward_elimination_v1_1(Y, X, alpha):
    current_results = sm.OLS(Y,X).fit()
    remaining_regressors=list(range(np.shape(X)[1]))
    while len(remaining_regressors) > 0:
        max_pvalue_pos = np.argmax(current_results.pvalues)
        if current_results.pvalues[max_pvalue_pos] > alpha:
            #delete it from remaining_regressors
            X = np.delete(X, max_pvalue_pos, 1)
            current_results = sm.OLS(Y,X).fit()
            remaining_regressors.pop(max_pvalue_pos)
        else:
            return remaining_regressors, current_results
    
    return remaining_regressors, current_results
    

In [13]:
#this version requires that const stays in the model
def backward_elimination_v1_2(Y, X, alpha):
    current_results = sm.OLS(Y,X).fit()
    remaining_regressors=list(range(np.shape(X)[1]))
    while len(remaining_regressors) > 0:
        pvals = current_results.pvalues
        pvals = np.delete(pvals,0)
        max_pvalue_pos = np.argmax(pvals) + 1
        if max_pvalue_pos == 0:
            print('error, should never be pos 0')
        if current_results.pvalues[max_pvalue_pos] > alpha:
            #delete it from remaining_regressors
            X = np.delete(X, max_pvalue_pos, 1)
            current_results = sm.OLS(Y,X).fit()
            remaining_regressors.pop(max_pvalue_pos)
        else:
            return remaining_regressors, current_results
        
    return remaining_regressors, current_results
    

In [7]:
df  = pd.read_excel('NFL_data.xlsx', sheet_name = "Data")
df_n = df.to_numpy()
X = df_n[:, 1:(df_n.shape[1])]
Y = df_n[:, [0]]
#add a column of 1's to the front to account for the constant
X = sm.add_constant(X)

In [15]:
rem_reg, results_with_const = backward_elimination_v1_2(Y, X, 0.1)
rem_reg, results_without_const = backward_elimination_v1_1(Y, X, 0.1)

In [16]:
print(results_without_const.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.958
Model:                            OLS   Adj. R-squared (uncentered):              0.953
Method:                 Least Squares   F-statistic:                              192.2
Date:                Fri, 07 Jan 2022   Prob (F-statistic):                    2.18e-17
Time:                        18:42:26   Log-Likelihood:                         -52.563
No. Observations:                  28   AIC:                                      111.1
Df Residuals:                      25   BIC:                                      115.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [17]:
print(results_with_const.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     29.44
Date:                Fri, 07 Jan 2022   Prob (F-statistic):           3.27e-08
Time:                        18:42:39   Log-Likelihood:                -52.532
No. Observations:                  28   AIC:                             113.1
Df Residuals:                      24   BIC:                             118.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8084      7.901     -0.229      0.8